In [ ]:
!pip uninstall paddlepaddle

Found existing installation: paddlepaddle 2.3.1
Uninstalling paddlepaddle-2.3.1:
  Would remove:
    /home/linx123-rtx/anaconda3/envs/paddleseg/bin/fleetrun
    /home/linx123-rtx/anaconda3/envs/paddleseg/bin/paddle
    /home/linx123-rtx/anaconda3/envs/paddleseg/lib/python3.10/site-packages/_foo.cpython-310-x86_64-linux-gnu.so
    /home/linx123-rtx/anaconda3/envs/paddleseg/lib/python3.10/site-packages/paddle/*
    /home/linx123-rtx/anaconda3/envs/paddleseg/lib/python3.10/site-packages/paddlepaddle-2.3.1.dist-info/*
  Would not remove (might be manually added):
    /home/linx123-rtx/anaconda3/envs/paddleseg/lib/python3.10/site-packages/paddle/include/third_party/externalError/data/externalErrorMsg.pb
Proceed (Y/n)? 

In [2]:
# tutorial API: https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.6/docs/api_example.md
from paddleseg.models import PPLiteSeg

In [11]:
from paddleseg.models.backbones import STDC1
model = PPLiteSeg(num_classes=2,
                  arm_out_chs=[32,64,]
                  backbone=STDC1(),
                 pretrained=None)

In [19]:
# create random train,val,test distribution of dataset
import os
import glob
from random import shuffle
dataset_root = "/media/linx123-rtx/Elements/Drone_Vision_Datasets/Hannasscapes_small/"
images = glob.glob(os.path.join(dataset_root,"texture","*.png"))
shuffle(images)
n = len(images)
sets = ["train","val","test"]
train_images = images[:(n//100*75)]
val_images = images[(n//100*75):n//100*90]
test_images = images[(n//100*90):]
im_sets = {"train":train_images,"val":val_images,"test":test_images}

for s in sets:
    lines = []
    for image in im_sets[s]:
        image_name = image.split("/")[-1]
        line = f"texture/{image_name} annotations/{image_name}\n"
        lines.append(line)
    with open(os.path.join(dataset_root,f"{s}_list.txt"), "w") as f:
        f.writelines(lines)

In [22]:
# build transforms for training
import paddleseg.transforms as T
transforms = [
    T.Resize(target_size=(1024,512)),
    T.RandomHorizontalFlip(),
    T.Normalize()
]
# build train dataset: https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.6/docs/apis/datasets/datasets.md#custom-dataset
from paddleseg.datasets import Dataset
dataset_root = "/media/linx123-rtx/Elements/Drone_Vision_Datasets/Hannasscapes_small/"
train_path = "/media/linx123-rtx/Elements/Drone_Vision_Datasets/Hannasscapes_small/train_list.txt"
val_path = "/media/linx123-rtx/Elements/Drone_Vision_Datasets/Hannasscapes_small/val_list.txt"
num_classes = 2

dataset = Dataset(transforms = transforms,
                 dataset_root=dataset_root,
                 num_classes=num_classes,
                 train_path= train_path,
                 mode = "train")

transforms = [
    T.Resize(target_size=(1024,512)),
    T.Normalize()
]
# build validation dataset
val_dataset = Dataset(dataset_root=dataset_root,
                      transforms = transforms,
                      num_classes=num_classes,
                      val_path=val_path,
                      mode = "val")

In [23]:
# optimizer 
import paddle
base_lr = 0.005
lr = paddle.optimizer.lr.PolynomialDecay(base_lr,power=0.9,decay_steps=1000,end_lr=0)

optimizer = paddle.optimizer.Momentum(lr,parameters=model.parameters(),momentum=0.9,weight_decay=4.0e-5)

In [31]:
# floss function
from paddleseg.models.losses import OhemCrossEntropyLoss

losses = {}
losses["types"] = [OhemCrossEntropyLoss(min_kept=130000)] * 3
losses["coef"] = [1, 1, 1]

In [1]:
import os
os.system("export CUDA_VISIBLE_DEVICES=0")

0

In [ ]:
# TRAIN
import os
os.system('''python train.py \
--config /home/linx123-rtx/PaddleSeg/configs/pp_liteseg/pp_liteseg_stdc1_hannasscapes_small_1024x512_scale0.5_160k.yml \
--do_eval \
--use_vdl \
--save_interval 500 \
--save_dir output/pp_liteset_hannascapes''')

/home/linx123-rtx/PaddleSeg/paddleseg/models/losses/decoupledsegnet_relax_boundary_loss.py:19: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift
/home/linx123-rtx/PaddleSeg/paddleseg/transforms/functional.py:18: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import distance_transform_edt


2022-08-04 20:03:50 [INFO]	
------------Environment Information-------------
platform: Linux-5.4.0-122-generic-x86_64-with-glibc2.31
Python: 3.10.4 (main, Mar 31 2022, 08:41:55) [GCC 7.5.0]
Paddle compiled with cuda: False
GCC: gcc (Ubuntu 11.1.0-1ubuntu1~20.04) 11.1.0
PaddleSeg: 2.5.0
PaddlePaddle: 2.3.1
OpenCV: 4.6.0
------------------------------------------------
[INFO] set device to cpu
2022-08-04 20:03:50 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 160000
loss:
  coef:
  - 1
  - 1
  - 1
  types:
  - ignore_index: 255
    min_kept: 130000
    type: OhemCrossEntropyLoss
  - ignore_index: 255
    min_kept: 130000
    type: OhemCrossEntropyLoss
  - ignore_index: 255
    min_kept: 130000
    type: OhemCrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.005
  power: 0.9
  type: PolynomialDecay
  warmup_iters: 1000
  warmup_start_lr: 1.0e-05
model:
  arm_out_chs:
  - 32
  - 64
  - 128
  backbone:
    pretrained: https://bj.bcebos.com/paddleseg

/home/linx123-rtx/anaconda3/envs/paddleseg/lib/python3.10/site-packages/paddle/nn/layer/norm.py:653: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(
/home/linx123-rtx/anaconda3/envs/paddleseg/lib/python3.10/site-packages/paddle/fluid/dygraph/math_op_patch.py:276: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.int64, the right dtype will convert to paddle.float32
  warnings.warn(
